In [31]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import cv2
import json
import os
import time #for tracking
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

class chinPointDataset(Dataset):
    def __init__(self, imageFolder, annotFile, idealWidth, idealHeight):
        if(annotFile):
            self.imageFolder = imageFolder
            self.annotFile = annotFile
            t1=time.time()
            # count images
            self.size = 0
            for image in os.listdir(imageFolder):
                if image.endswith(".png"): #redundant
                    self.size += 1
            
            # open json
            with open(annotFile, 'r') as f:
                self.jsonAnnot = json.load(f)
            
            # preallocate arrays
            self.allImgsArray = np.zeros([self.size, idealHeight, idealWidth], dtype=np.float32)
            self.allAnnotsArray = np.zeros([self.size, 2], dtype=np.float32)
            
            # load all images and annotations

            # loop through and load all iamges in a folder. 
            #create a numpy array from these images.
            #then recolor it and resize it and app
            t2=time.time()
            for idx, image in enumerate(os.listdir(imageFolder)):
                if image.endswith(".png"): #redundant
                    
                    imgArr = cv2.imread(os.path.join(imageFolder, image))
                    height, width, channels = imgArr.shape
                    imgArr = cv2.cvtColor(imgArr, cv2.COLOR_BGR2GRAY)
                    imgArr = cv2.resize(imgArr, (idealWidth, idealHeight))  # cv2.resize takes (width, height)
                    
                    # normalize to 0-1 - crucial because nns generally work best with 0-1 normalized data... 
                    self.allImgsArray[idx] = imgArr.astype(np.float32) / 255.0
                    
                    # scale annotation from original coords to ideal coords
                    origX, origY = self.jsonAnnot[image]
                    
                    self.allAnnotsArray[idx] = [
                        origX * (idealWidth / width),
                        origY * (idealHeight / height)
                    ]
            
        else:
            t1=time.time()
            self.size = len(os.listdir(imageFolder))
            self.allImgsArray = np.zeros([self.size, idealHeight, idealWidth], dtype=np.float32)
            t2=time.time()
            for idx, image in enumerate(os.listdir(imageFolder)):
                if image.endswith(".png"): #redundant
                    imgArr = cv2.imread(os.path.join(imageFolder, image))
                    height, width, channels = imgArr.shape
                    imgArr = cv2.cvtColor(imgArr, cv2.COLOR_BGR2GRAY)
                    imgArr = cv2.resize(imgArr, (idealWidth, idealHeight))  # cv2.resize takes (width, height)
                    
                    # normalize to 0-1 - crucial because nns generally work best with 0-1 normalized data... 
                    self.allImgsArray[idx] = imgArr.astype(np.float32) / 255.0
        t3=time.time()
        print(f"Time taken for each stage:")
        print(f"Time taken for counting & loading annotations: {t2-t1} seconds")
        print(f"Time taken for loading & processing images: {t3-t2} seconds")
        print(f"Total time taken: {t3-t1} seconds")
    def __len__(self):
        return self.size
    
    def __getitem__(self, idx):
        # returns (image, annotation) as tensors
        img = torch.from_numpy(self.allImgsArray[idx]).unsqueeze(0)  # shape: [1, H, W] for single channel
        annot = torch.from_numpy(self.allAnnotsArray[idx])  # shape: [2]
        return img, annot

class chinPointNet(nn.Module):
    def __init__(self,ks=3):
        super().__init__()
        # simple conv stack -> regression head
        # input: [batch, 1, 96, 96]
        self.conv1 = nn.Conv2d(1, 32, kernel_size=ks, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=ks, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=ks, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()
        
        # after 3 pools: 96 -> 48 -> 24 -> 12
        # so final feature map is [batch, 128, 12, 12]
        self.fc1 = nn.Linear(128 * 12 * 12, 256)
        self.fc2 = nn.Linear(256, 2)  # output: [batch, 2] for (x, y)
    
    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        x = x.view(x.size(0), -1)  # flatten
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x



In [ ]:
def lrScheduler(baseLr, i):
    # formula: scales down, 10% of orig at 500 iters, 1% at 1000
    return baseLr * (10 ** (-i / 500))

imageFolder = "Imgs"
annotFile = "Imgs_Annotate.json"
testImageFolder = "testImgs"
idealWidth = 96
idealHeight = 96
chinPointDataset = chinPointDataset(imageFolder, annotFile, idealWidth, idealHeight)
testChinPointDataset = chinPointDataset(imageFolder, annotFile=None, idealWidth, idealHeight)
dataloader = DataLoader(
    chinPointDataset,
    batch_size=64,
    shuffle=True,
    drop_last=True,
    num_workers=0,              # ✅ Parallel data loading
    pin_memory=True,            # ✅ CRITICAL for fast GPU transfer!
    persistent_workers=False     # ✅ Reuse workers across epochs
)
testDataloader = DataLoader(
    testChinPointDataset,
    batch_size=36,
    shuffle=True,
    drop_last=True,
    num_workers=0,
    pin_memory=True,
    persistent_workers=False
)
# Initialize model ON GPU
model = chinPointNet().to("cuda") 

# Loss function (can stay on CPU, will auto-move)
criterion = nn.MSELoss()

# Optimizer
baseLr = 0.001
optimizer = optim.SGD(model.parameters(), lr=baseLr)  # Added momentum




SyntaxError: positional argument follows keyword argument (3990684854.py, line 11)

In [ ]:
ROOT_PATH = 'Imgs'
ANNOT_PATH = 'Imgs_Annotate.json'
OUTPUT_PATH = 'trainPredPlots'
BATCH_SIZE = 64
def savePreds(preds, label, it, da, def_size=96,outputPath=OUTPUT_PATH):
    # for keys, values in imgAnnot.items():
    #     if keys == imgAnnot
    global BATCH_SIZE
    print('saving...')
    data = da.to('cpu').squeeze(0).numpy()
    print(data.shape)
    data = data.transpose(0,2,3,1)
    # plt.imshow(data[0])
    # plt.plot()
    # plt.show()
    iter = it
    print("PREDS:\n",preds)
       # Multiply by def_size and round to nearest integer
    Opreds = torch.round(preds)
    if(label):
        Olabel = torch.round(label)
    else:
        Olabel = None
    #print("Lookie here: \n",Opreds, Olabel)
    
    # print("Rounded predictions:", Opreds)
    # print("Rounded labels:", Olabel)

    fig, axes = plt.subplots(5, 5, figsize=(36, 36))
    for i, ax in enumerate(axes.flat):
        fileName = f"{i+iter:06ed}.png"

    # Directly extract and convert to float from tensors
        xp, yp = Opreds[i][0].item(), Opreds[i][1].item()
        if(label):
            xl, yl = Olabel[i][0].item(), Olabel[i][1].item()
        else:
            xl, yl = None, None
        # print(xp,yp)
        # print(xl,yl)

        # print(f"Coordinates (Preds): x={xp}, y={yp}")
        # print(f"Coordinates (Label): x={xl}, y={yl}")
        if xp <= 0:
            xp = 0
        if yp <= 0:
            yp = 0
        
        #for file in os.listdir(imgPath):
        #full_path = os.path.join(imgPath, fileName)
        new_path = os.path.join(OUTPUT_PATH, fileName)
        img = data[i]
        ax.imshow(img,cmap='gray')
        ax.axis('off')  # Turn off the axis for a cleaner look
        ax.set_title(f'Image {i + 1}')  # Set the title for each subplot
        
        p = plt.Circle((xp, yp), 3, color='red')
        if(label):
            l = plt.Circle((xl, yl), 3, color='blue')
            ax.add_patch(l)
        ax.add_patch(p)

    fileName = f"{(iter*BATCH_SIZE):06d}_{((iter+1)*BATCH_SIZE):06d}.png"
    plt.tight_layout()
    new_path = os.path.join(OUTPUT_PATH, fileName)
    plt.savefig(new_path)

    #plt.show()
    
    plt.close()

In [30]:
scaler = torch.amp.GradScaler("cuda")  # GradScaler for CUDA AMP

if __name__ == "__main__":
    # training loop
    for epoch in range(1000):
        for batchIdx, (batchImg, batchGTAnnot) in enumerate(dataloader):
            # forward
            batchImg = batchImg.to("cuda", non_blocking=True)
            batchGTAnnot = batchGTAnnot.to("cuda", non_blocking=True)
            batchOutAnnot = model(batchImg)
            loss = criterion(batchOutAnnot, batchGTAnnot)
            optimizer.zero_grad(set_to_none=True)
            with torch.autocast(device_type="cuda", dtype=torch.float16):
                batchOutAnnot = model(batchImg)
                loss = criterion(batchOutAnnot, batchGTAnnot)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
        newLr = lrScheduler(baseLr, epoch)
        for pg in optimizer.param_groups:
            pg["lr"] = newLr
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.6f}, LR: {newLr:.6f}")
            savePreds(batchOutAnnot,batchGTAnnot,epoch,batchImg)
            testImg = next(iter(testDataloader))
            testImg = testImg.to("cuda", non_blocking=True)
            testOutAnnot = model(testImg)
            savePreds(testOutAnnot,label=None,epoch,testImg,outputPath="testPredPlots")
        if epoch % 50 == 0:
            torch.save(model.state_dict(), f"modelCheckpoints/epoch_{epoch}.pth")

SyntaxError: positional argument follows keyword argument (1233770962.py, line 30)

In [76]:
# # === TESTING chinPointDataset.__getitem__() ===

# print("="*70)
# print("TESTING DATASET __getitem__ METHOD")
# print("="*70)

# # Method 1: Using bracket notation (preferred)
# print("\n--- Method 1: Bracket notation ---")
# img, annot = chinPointDataset[0]  # This calls __getitem__(0)
# print(f"Image shape: {img.shape}")
# print(f"Image dtype: {img.dtype}")
# print(f"Image min/max values: {img.min():.3f} / {img.max():.3f}")
# print(f"Annotation: {annot}")
# print(f"Annotation shape: {annot.shape}")

# # Method 2: Calling __getitem__ directly (less common)
# print("\n--- Method 2: Direct call (equivalent) ---")
# img2, annot2 = chinPointDataset.__getitem__(0)
# print(f"Same result: {torch.equal(img, img2)}")

# # Test multiple samples
# print("\n--- Testing multiple samples ---")
# for i in range(min(5, len(chinPointDataset))):
#     img, annot = chinPointDataset[i]
#     print(f"Sample {i}: img shape={img.shape}, annot={annot.numpy()}")

# # Test random samples
# print("\n--- Testing random samples ---")
# for _ in range(3):
#     idx = random.randint(0, len(chinPointDataset) - 1)
#     img, annot = chinPointDataset[idx]
#     print(f"Random idx {idx}: annot point = ({annot[0]:.2f}, {annot[1]:.2f})")



# # Visualize a sample
# print("\n--- Visualizing sample 0 ---")
# img, annot = chinPointDataset[0]
# plt.figure(figsize=(8, 8))
# plt.imshow(img.squeeze(), cmap='gray')
# plt.scatter(annot[0].item(), annot[1].item(), c='red', s=100, marker='x', linewidths=3)
# plt.title(f'Sample 0: Chin Point at ({annot[0]:.1f}, {annot[1]:.1f})')
# plt.axis('off')
# plt.tight_layout()
# plt.show()

# print(f"\nTotal dataset size: {len(chinPointDataset)}")

In [77]:
# """
# =====================================================================
# PYTORCH DATALOADER COMPREHENSIVE TUTORIAL
# Web-backed best practices for 2025
# =====================================================================
# """

# import torch
# from torch.utils.data import DataLoader
# import time

# print("="*70)
# print("PART 1: BASIC DATALOADER USAGE")
# print("="*70)

# # Create a basic DataLoader
# basicLoader = DataLoader(
#     dataset=chinPointDataset,
#     batch_size=32,
#     shuffle=True
# )

# print(f"\nDataLoader created!")
# print(f"Total samples: {len(chinPointDataset)}")
# print(f"Batch size: {basicLoader.batch_size}")
# print(f"Number of batches: {len(basicLoader)}")

# # Iterate through the DataLoader
# print("\n--- First 3 batches ---")
# for batchIdx, (images, annotations) in enumerate(basicLoader):
#     if batchIdx < 3:
#         print(f"Batch {batchIdx}:")
#         print(f"  Images shape: {images.shape}")  # [batch_size, 1, H, W]
#         print(f"  Annotations shape: {annotations.shape}")  # [batch_size, 2]
#         print(f"  First annotation in batch: {annotations[0]}")
#     else:
#         break

# print("\n" + "="*70)
# print("PART 2: KEY DATALOADER PARAMETERS")
# print("="*70)

# # 2.1 batch_size
# print("\n--- 2.1 BATCH SIZE ---")
# print("Controls how many samples per batch")
# print("Larger = faster training but more memory")
# print("Common values: 16, 32, 64, 128, 256")

# loaderSmallBatch = DataLoader(chinPointDataset, batch_size=8)
# loaderLargeBatch = DataLoader(chinPointDataset, batch_size=64)
# print(f"Small batch (8): {len(loaderSmallBatch)} batches")
# print(f"Large batch (64): {len(loaderLargeBatch)} batches")

# # 2.2 shuffle
# print("\n--- 2.2 SHUFFLE ---")
# print("Randomizes sample order each epoch")
# print("CRITICAL for training! Prevents learning data order")

# loaderNoShuffle = DataLoader(chinPointDataset, batch_size=4, shuffle=False)
# loaderWithShuffle = DataLoader(chinPointDataset, batch_size=4, shuffle=True)

# print("\nNo shuffle (first batch indices in order):")
# for i, (img, annot) in enumerate(loaderNoShuffle):
#     if i == 0:
#         print(f"  First sample annot: {annot[0]}")
#     break

# print("\nWith shuffle (randomized):")
# for i, (img, annot) in enumerate(loaderWithShuffle):
#     if i == 0:
#         print(f"  First sample annot: {annot[0]}")
#     break

# # 2.3 num_workers
# print("\n--- 2.3 NUM_WORKERS (Parallel Data Loading) ---")
# print("Uses multiple processes to load data faster")
# print("Best Practice: Start with 4, increase if I/O bound")
# print("Warning: On Windows, num_workers can cause issues")
# print("         Use 0 for debugging, 2-4 for training")

# # Test with different worker counts
# batchSize = 32
# testSize = min(len(chinPointDataset), 500)

# for numWorkers in [0, 2]:
#     testLoader = DataLoader(
#         chinPointDataset,
#         batch_size=batchSize,
#         shuffle=True,
#         num_workers=numWorkers
#     )
    
#     start = time.time()
#     for i, (img, annot) in enumerate(testLoader):
#         if i >= testSize // batchSize:
#             break
#     elapsed = time.time() - start
    
#     print(f"num_workers={numWorkers}: {elapsed:.3f}s for {testSize} samples")

# # 2.4 drop_last
# print("\n--- 2.4 DROP_LAST ---")
# print("Drops the last incomplete batch if dataset size not divisible by batch_size")

# loaderKeepLast = DataLoader(chinPointDataset, batch_size=32, drop_last=False)
# loaderDropLast = DataLoader(chinPointDataset, batch_size=32, drop_last=True)

# print(f"Dataset size: {len(chinPointDataset)}")
# print(f"Keep last: {len(loaderKeepLast)} batches")
# print(f"Drop last: {len(loaderDropLast)} batches")

# # 2.5 pin_memory
# print("\n--- 2.5 PIN_MEMORY ---")
# print("Speeds up CPU→GPU transfer")
# print("Set to True if using CUDA")

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(f"Current device: {device}")

# loaderPinned = DataLoader(
#     chinPointDataset,
#     batch_size=32,
#     pin_memory=True if torch.cuda.is_available() else False
# )
# print(f"Pin memory enabled: {loaderPinned.pin_memory}")

# print("\n" + "="*70)
# print("PART 3: REAL TRAINING SCENARIO")
# print("="*70)

# # Split dataset into train/val (80/20 split)
# datasetSize = len(chinPointDataset)
# trainSize = int(0.8 * datasetSize)
# valSize = datasetSize - trainSize

# print(f"\nDataset split:")
# print(f"  Total: {datasetSize}")
# print(f"  Train: {trainSize} ({trainSize/datasetSize*100:.1f}%)")
# print(f"  Val: {valSize} ({valSize/datasetSize*100:.1f}%)")

# # Use random_split to split dataset
# from torch.utils.data import random_split

# trainDataset, valDataset = random_split(
#     chinPointDataset,
#     [trainSize, valSize],
#     generator=torch.Generator().manual_seed(42)  # For reproducibility
# )

# # Create DataLoaders with best practices
# trainLoader = DataLoader(
#     trainDataset,
#     batch_size=32,
#     shuffle=True,          # IMPORTANT: Shuffle training data
#     num_workers=2,         # Parallel loading
#     pin_memory=torch.cuda.is_available(),  # Fast GPU transfer
#     drop_last=True         # Consistent batch sizes
# )

# valLoader = DataLoader(
#     valDataset,
#     batch_size=64,          # Can use larger batch for validation
#     shuffle=False,          # NO shuffle for validation
#     num_workers=2,
#     pin_memory=torch.cuda.is_available(),
#     drop_last=False         # Keep all validation data
# )

# print(f"\nTrain loader: {len(trainLoader)} batches")
# print(f"Val loader: {len(valLoader)} batches")

# # Simulate training loop
# print("\n--- Simulated training loop ---")
# numEpochs = 2

# for epoch in range(numEpochs):
#     print(f"\nEpoch {epoch+1}/{numEpochs}")
    
#     # Training
#     trainLoss = 0
#     for batchIdx, (images, annotations) in enumerate(trainLoader):
#         # Move to device (GPU if available)
#         images = images.to(device)
#         annotations = annotations.to(device)
        
#         # Your model forward pass would go here
#         # loss = model(images, annotations)
        
#         if batchIdx < 2:  # Print first 2 batches
#             print(f"  Train batch {batchIdx}: images {images.shape}, annots {annotations.shape}")
    
#     # Validation
#     with torch.no_grad():
#         valLoss = 0
#         for batchIdx, (images, annotations) in enumerate(valLoader):
#             images = images.to(device)
#             annotations = annotations.to(device)
            
#             # Your model evaluation would go here
            
#             if batchIdx < 2:
#                 print(f"  Val batch {batchIdx}: images {images.shape}, annots {annotations.shape}")

# print("\n" + "="*70)
# print("PART 4: ADVANCED TIPS & BEST PRACTICES")
# print("="*70)

# print("""
# 🎯 BEST PRACTICES SUMMARY (2025):

# 1. BATCH SIZE:
#    - Start with 32, increase if you have GPU memory
#    - Powers of 2 (16, 32, 64, 128) often work best
#    - Smaller batches = noisier gradients but better generalization

# 2. SHUFFLE:
#    - ALWAYS True for training
#    - ALWAYS False for validation/testing
#    - Prevents model from learning data order

# 3. NUM_WORKERS:
#    - Windows: Use 0 for debugging, 2-4 for training
#    - Linux/Mac: Can use more (4-8)
#    - Too many workers = slower due to overhead
#    - Test different values with your specific dataset

# 4. PIN_MEMORY:
#    - Set to True if using GPU
#    - Speeds up host→device transfer
#    - Uses more CPU memory

# 5. DROP_LAST:
#    - True for training (consistent batch sizes help some models)
#    - False for validation (don't lose data)

# 6. PERSISTENT_WORKERS (Advanced):
#    - Set to True to reuse workers across epochs
#    - Faster but uses more memory
#    - Only works if num_workers > 0

# 7. PREFETCH_FACTOR (Advanced):
#    - Controls how many batches each worker preloads
#    - Default is 2 (good for most cases)

# ⚡ PERFORMANCE TIPS:
# - Preload all data into memory if possible (like your dataset does!)
# - Use transforms in __getitem__ for on-the-fly augmentation
# - Profile your code to find bottlenecks
# - Consider using torch.compile() for the model (PyTorch 2.0+)

# ⚠️ COMMON MISTAKES:
# ❌ Shuffling validation data
# ❌ Different batch sizes between epochs
# ❌ Not using pin_memory with GPU
# ❌ Too many num_workers causing overhead
# ❌ Not dropping last batch when model requires fixed batch size

# ✅ TYPICAL CONFIGURATION:
# """)

# print("\n# For TRAINING:")
# print("""trainLoader = DataLoader(
#     dataset=trainDataset,
#     batch_size=32,
#     shuffle=True,
#     num_workers=2,
#     pin_memory=torch.cuda.is_available(),
#     drop_last=True
# )""")

# print("\n# For VALIDATION/TESTING:")
# print("""valLoader = DataLoader(
#     dataset=valDataset,
#     batch_size=64,
#     shuffle=False,
#     num_workers=2,
#     pin_memory=torch.cuda.is_available(),
#     drop_last=False
# )""")

# print("\n" + "="*70)
# print("DATALOADER TUTORIAL COMPLETE!")
# print("="*70)

In [24]:
d


{'a': 1, 'b': 2}

In [25]:
d.pop("b")

2

In [27]:
d

{'a': 1}

In [29]:
d[]"b")

AttributeError: 'dict' object has no attribute 'insert'

In [ ]:
#How dictionaries work in python:

#d.update() #overrides values and inserts new ones whereever they do not yet exist. Does it do this regardless of order?
dict = {"a":1, "b":2,"ca":4}
dict.get("key") #returns none if key is DNE, else its value.
dict.get("item",1) #returns returnvalue (1 is current returnvalue) if key is DNE, else the key's value.
#enumerate() splits a list or dict into an index and the value.. by default for a dictionary this results in, if idx,item is done, the item being the key.
#just doing dict.copy() passes pointers to the data to a new instance allocated in memory, aka a shallow copy. But deep copying actually copies the 
# data itself to a new object

1

In [45]:
print(dict.get("ca"))

4


In [79]:
d.get('z', 1.3**7)
print(26**5)

11881376


In [62]:
l=[5,6,44,1]
for idx,f in enumerate(dict):
    print(idx," and: " + f)

0  and: a
1  and: b
2  and: ca
